In [2]:
from qiskit import IBMQ
import numpy as np
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

from qiskit import pulse                  # This is where we access all of our Pulse features!
from qiskit.circuit import Parameter      # This is Parameter Class for variable parameters.
from qiskit.circuit import QuantumCircuit, Gate
from qiskit.tools.monitor import job_monitor

from datetime import datetime

IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q', group='open', project='main')
backend = provider.get_backend('ibm_oslo')
backend_config = backend.configuration()
dt = backend_config.dt

backend.configuration().timing_constraints
acquire_alignment = backend.configuration().timing_constraints['acquire_alignment']
granularity = backend.configuration().timing_constraints['granularity']
pulse_alignment = backend.configuration().timing_constraints['pulse_alignment']
lcm = np.lcm(acquire_alignment, pulse_alignment)
print(f"Least common multiple of acquire_alignment and pulse_alignment: {lcm}")
backend_defaults = backend.defaults()

# unit conversion factors -> all backend properties returned in SI (Hz, sec, etc.)
GHz = 1.0e9 # Gigahertz
MHz = 1.0e6 # Megahertz
us = 1.0e-6 # Microseconds
ns = 1.0e-9 # Nanoseconds

# We will find the qubit frequency for the following qubit.
qubit = 0

x_duration = 320
x_sigma = 80
f01 = backend_defaults.qubit_freq_est[qubit]
f12 = f01 + backend_properties.qubits[qubit][3].value*GHz

def get_closest_multiple_of(vaule, base_number):
    return int(vaule + base_number/2) - (int(vaule + base_number/2) % base_number)
# samples need to be multiples of 16
def get_closest_multiple_of_16(num):
    return get_closest_multiple_of(num, granularity)
# Convert seconds to dt
def get_dt_from(sec):
    return get_closest_multiple_of(sec/dt, lcm)

scale_factor = 1e-7
def get_job_data(job, average):
    """Retrieve data from a job that has already run.
    Args:
        job (Job): The job whose data you want.
        average (bool): If True, gets the data assuming data is an average.
                        If False, gets the data assuming it is for single shots.
    Return:
        list: List containing job result data.
    """
    job_results = job.result()  # timeout parameter set to 120 s
    result_data = []
    for i in range(len(job_results.results)):
        if average:  # get avg data
            result_data.append(np.real(job_results.get_memory(i)[qubit] * scale_factor))
            print('here')
        else:  # get single data
            result_data.append(job_results.get_memory(i)[:, qubit] * scale_factor)
            print(i, end='\r')
    return result_data

def fit_function(x_values, y_values, function, init_params):
    fitparams, conv = curve_fit(function, x_values, y_values, init_params)
    y_fit = function(x_values, *fitparams)
    
    return fitparams, y_fit

def baseline_remove(values):
    return np.array(values) - np.mean(values)

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


Least common multiple of acquire_alignment and pulse_alignment: 16
Traceback (most recent call last):
  Input In [2] in <cell line: 39>
    f12 = f01 + backend_properties.qubits[qubit][3].value*GHz
NameError: name 'backend_properties' is not defined

Use %tb to get the full traceback.


In [9]:
def rabi01():
    num_rabi_points = 100
    drive_amp_min = 0
    drive_amp_max = 0.75
    drive_amps = np.linspace(drive_amp_min, drive_amp_max, num_rabi_points)
    f01 = backend_defaults.qubit_freq_est[qubit]
    
    drive_amp = Parameter('drive_amp')
    with pulse.build(backend=backend, default_alignment='sequential', name='Rabi Experiment') as rabi_sched:
        drive_duration = 320
        drive_sigma = 80
        drive_chan = pulse.drive_channel(qubit)
        pulse.set_frequency(f01, drive_chan)
        pulse.play(pulse.Gaussian(duration=drive_duration,
                                  amp=drive_amp,
                                  sigma=drive_sigma,
                              name='Rabi Pulse'), drive_chan)
        
    rabi_gate = Gate("rabi", 1, [drive_amp])
    qc_rabi = QuantumCircuit(1, 1)
    qc_rabi.append(rabi_gate, [0])
    qc_rabi.measure(0, 0)
    qc_rabi.add_calibration(rabi_gate, (0,), rabi_sched, [drive_amp])
    exp_rabi_circs = [qc_rabi.assign_parameters({drive_amp: a}, inplace=False) for a in drive_amps]
    num_shots_per_point = 20000
    
    rabi01_job = backend.run(exp_rabi_circs, 
                      meas_level=1,
                      meas_return='avg',
                      shots=num_shots_per_point)
    print('Rabi 01 job id:', rabi01_job.job_id())
    job_monitor(rabi01_job)
    time = now.strftime("%d/%m/%Y %H:%M:%S")
    print("Job complete at", time)	
    rabi_results = rabi01_job.result()
    rabi_values = []
    for i in range(num_rabi_points):
        rabi_values.append(rabi_results.get_memory(i)[0] * scale_factor)
        
    
    fit_params, y_fit = fit_function(drive_amps,
                                 rabi_values, 
                                 lambda x, A, B, drive_period, phi: (A*np.cos(2*np.pi*x/drive_period - phi) + B),
                                 [15, 0, 0.15, np.pi])
    drive_period = fit_params[2]
    X01_amp = abs(drive_period / 2)
    plt.scatter(drive_amps, rabi_values, color='black')
    plt.plot(drive_amps, y_fit, color='red')
        
    plt.axvline(X01_amp, color='red', linestyle='--')
    plt.axvline(drive_period, color='red', linestyle='--')
    plt.annotate("", xy=(drive_period, 0), xytext=(drive_period/2,0), arrowprops=dict(arrowstyle="<->", color='red'))
    plt.annotate("$\pi$", xy=(drive_period/2-0.03, 0.1), color='red')

    plt.xlabel("Drive amp [a.u.]", fontsize=15)
    plt.ylabel("Measured signal [a.u.]", fontsize=15)
    plt.show()

    
    print(f"Pi Amplitude = {X01_amp}")
    return X01_amp

def rabi12(x01_amp):    
    with pulse.build(backend=backend, default_alignment='sequential', name=r'$X_{\pi}^{(01)}$') as X01_Gaussian_sched:
    drive_chan = pulse.drive_channel(qubit)
    pulse.set_frequency(f01, drive_chan)
    pulse.play(pulse.Gaussian(duration=x_duration,
                            amp=x01_amp,
                            sigma=x_sigma,
                            name=r'$X_{\pi}^{(01)}$'), drive_chan)
    num_rabi_points = 100 # number of experiments (ie amplitudes to sweep out)

    drive_amp_min = 0
    drive_amp_max = 0.4
    drive_amps = np.linspace(drive_amp_min, drive_amp_max, num_rabi_points)
    
    amp = Parameter('amp')
    with pulse.build(backend=backend, default_alignment='sequential', name='Amp sweep') as rabi_sched:
        drive_chan = pulse.drive_channel(qubit)
        pulse.set_frequency(f12, drive_chan)
        pulse.play(pulse.Gaussian(duration=x_duration,
                                  amp=amp,
                                  sigma=x_sigma,
                                  name='x12_pulse'), drive_chan)
    rabi12_job = backend.run(exp_rabi_circs, 
                          meas_level=1, 
                          meas_return='avg', 
                          shots=20000)
    print('Rabi 12 job id:', rabi12_job.job_id())
    job_monitor(rabi12_job)
    time = now.strftime("%d/%m/%Y %H:%M:%S")
    print("Job complete at", time)
    
    rabi12_data = get_job_data(rabi12_job, average=True)
    rabi12_data = np.real(baseline_remove(rabi12_data))
    (rabi_12_fit_params, 
     rabi_12_y_fit) = fit_function(drive_amps,
                                   rabi12_data, 
                                   lambda x, A, B, drive_12_period, phi: (A*np.cos(2*np.pi*x/drive_12_period - phi) + B),
                                   [2, 1, 0.15, 0])

    plt.scatter(drive_amps, rabi12_data, color='black')
    plt.plot(np.linspace(drive_amps[0], drive_amps[-1], 10000), rabi_12_y_fit, color='red')

    drive_12_period = rabi_12_fit_params[2] 

    x12_amp = drive_12_period/2

    plt.axvline(x12_amp, color='red', linestyle='--')
    plt.axvline(x12_amp+drive_12_period/2, color='red', linestyle='--')
    plt.annotate("", xy=(pi_amp_12+drive_12_period/2, 0), xytext=(x12_amp,0), arrowprops=dict(arrowstyle="<->", color='red'))
    plt.annotate("$\pi$", xy=(x12_amp-0.03, 0.1), color='red')

    plt.xlabel("Drive amp [a.u.]", fontsize=15)
    plt.ylabel("Measured signal [a.u.]", fontsize=15)
    plt.title('Rabi Experiment (1->2)', fontsize=20)
    plt.show()
    print(f"Pi Amplitude (1->2) = {x12_amp}")
    return x12_amp

In [ ]:
x01_amp = rabi01()

Rabi 01 job id: 636bd3c6c18278ba3476cee9
Job Status: job is queued (48)    

In [ ]:
x12_amp = rabi12(x01_amp)